# Metodología de privatización

## Manipulación de datos

Primero se carga la base que nos suministro el ministerio de salud

In [2]:
import pickle
import pandas as pd

#Mac_David
file_path = '/Users/davidmoreno/Library/CloudStorage/OneDrive-SharedLibraries-Universidaddelosandes/David Stiven Romero Grajales - Trabajo de Grado/Trabajo de Grado 1/Base de datos y descripción/Entrenamiento_cronicas_N.pkl'

#archivo_pickle = 'Entrenamiento_cronicas_N.pkl'

with open(file_path, 'rb') as file:
    df = pickle.load(file)


Primero vamos a ver cómo están distribuidos los datos, y cuántos na tenemos en cada columna para ver que tanto tenemos que limpiar la base de datos

In [7]:
# Count the number of NaNs, zeros, and ones for each column
nan_counts = df.isna().sum()
zero_counts = (df == 0).sum()
one_counts = (df == 1).sum()

# Calculate the percentage of ones for each column
one_percentage = (one_counts / len(df)) * 100

# Create a dataframe to display the counts of NaNs, zeros, ones, and percentage of ones
zero_nan_summary = pd.DataFrame({
    'NaN_Count': nan_counts,
    'Zero_Count': zero_counts,
    'One_Count': one_counts,
    'One_Percentage': one_percentage
})

# Display the result
print(zero_nan_summary)


                                       NaN_Count  Zero_Count  One_Count  \
EPS                                            0           0          0   
NUM_IDE                                        0           0          0   
FECHA_NACI                                     0           0          0   
SEXO                                           0           0          0   
DPTO                                           0           0          0   
MUNI                                           0           0          0   
DIAS_COMP                                      0           0          0   
V8                                             0           0          0   
id                                             0           0          0   
Valor_Neto_Chunk                               0      996234          0   
Valor_Neto                                     0      357718         28   
DIAS_ESTAN_Neto_Chunk                          0      996234          0   
DIAS_ESTAN_Neto          

In [4]:
df.columns

Index(['EPS', 'NUM_IDE', 'FECHA_NACI', 'SEXO', 'DPTO', 'MUNI', 'DIAS_COMP',
       'V8', 'id', 'Valor_Neto_Chunk', 'Valor_Neto', 'DIAS_ESTAN_Neto_Chunk',
       'DIAS_ESTAN_Neto', 'Edad', 'SIDA_VIH', 'TUBERCULOSIS',
       'CANCER_OTROSCANCER', 'CANCER_ORGANOSDIGESTIVOS',
       'CANCER_ORGANOSRESPIRATORIOS', 'CANCER_MELANOMAYDELAPIEL',
       'CANCER_MAMA', 'CANCER_OTROSGENITALESFEMENINOS',
       'CANCER_CERVIXINVASIVO', 'CANCER_GENITALESMASCULINOS',
       'CANCER_TEJIDOLINFATICOYAFINES', 'CANCER_CERVIXINSITU', 'AUTOINMUNE',
       'DIABETES', 'SINDROMESCONVULSIVOS(EPILEPSIA)',
       'ENFERMEDADCARDIOVASCULAR_OTRA',
       'ENFERMEDADCARDIOVASCULAR_HIPERTENSION',
       'ENFERMEDADPULMONARDELARGADURACION', 'ASMA',
       'ARTRITISPIOGENASYREACTIVAS', 'ARTRITIS', 'ARTROSIS', 'RENAL_OTRA',
       'RENAL_RENALDELARGADURACION', 'TRANSPLANTE',
       'RENAL_INSUFICIENCIARENALOTRA', 'RENAL_INSUFICIENCIARENALCRONICA',
       'ANOMALIASGENETICASYCONGENITAS', 'CANCER_TERAPIACANCER', 'Grupo_

Se eliminan los datos donde todas las entradas son ceros para tener mayor efectividad en la privatización

In [5]:
# Eliminar columnas donde todas las entradas son cero
data_non_zero = df.loc[:, (df != 0).any(axis=0)]

Para mayor facilidad a la hora de manipular los datos de las enfermedades ya que vienen como binarias se concatenan separandolas por coma, facilitando la privatización

In [6]:

# Lista de todas las posibles columnas de enfermedades
columns_enfermedades = [
    'SIDA_VIH', 'TUBERCULOSIS', 'CANCER_OTROSCANCER', 'CANCER_ORGANOSDIGESTIVOS',
    'CANCER_ORGANOSRESPIRATORIOS', 'CANCER_MELANOMAYDELAPIEL', 'CANCER_MAMA',
    'CANCER_OTROSGENITALESFEMENINOS', 'CANCER_CERVIXINVASIVO', 'CANCER_GENITALESMASCULINOS',
    'CANCER_TEJIDOLINFATICOYAFINES', 'CANCER_CERVIXINSITU', 'AUTOINMUNE', 'DIABETES',
    'SINDROMESCONVULSIVOS(EPILEPSIA)', 'ENFERMEDADCARDIOVASCULAR_OTRA', 'ENFERMEDADCARDIOVASCULAR_HIPERTENSION',
    'ENFERMEDADPULMONARDELARGADURACION', 'ASMA', 'ARTRITISPIOGENASYREACTIVAS',
    'ARTRITIS', 'ARTROSIS', 'RENAL_OTRA', 'RENAL_RENALDELARGADURACION', 'TRANSPLANTE',
    'RENAL_INSUFICIENCIARENALOTRA', 'RENAL_INSUFICIENCIARENALCRONICA', 'ANOMALIASGENETICASYCONGENITAS',
    'CANCER_TERAPIACANCER'
]

# Crear la columna 'Enfermedad' que concatena los nombres de las enfermedades presentes, separadas por comas
data_non_zero['Enfermedad'] = (data_non_zero[columns_present] > 0).apply(lambda x: ', '.join(x.index[x]), axis=1)

# Mostrar las primeras filas para verificar la nueva columna 'Enfermedad'
data_non_zero[['Enfermedad']].head()


/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_72224/1138452748.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_non_zero['Enfermedad'] = (data_non_zero[columns_present] > 0).apply(lambda x: ', '.join(x.index[x]), axis=1)


,Enfermedad
rownames,
1184873,
1587467,ENFERMEDADCARDIOVASCULAR_OTRA
753062,
1802563,
1190692,


Para mejorar el entendimiento se debe reemplazar los 0 con Sin enfermedad 

In [7]:
data_non_zero.drop(columns=columns_enfermedades, inplace=True)

# Reemplazar valores vacíos o NaN en la columna 'Enfermedad' con 'Sin Enfermedad'
data_non_zero['Enfermedad'] = data_non_zero['Enfermedad'].apply(lambda x: 'Sin Enfermedad' if not x else x)


/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_72224/3052532982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_non_zero.drop(columns=columns_enfermedades, inplace=True)
/var/folders/tv/6r5469xx31vbksvh_943nnr1bcv0mh/T/ipykernel_72224/3052532982.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_non_zero['Enfermedad'] = data_non_zero['Enfermedad'].apply(lambda x: 'Sin Enfermedad' if not x else x)


finalmente se eliminan las variables que no nos brindaran información a futuro, notando que eliminamos fecha de nacimiento ya que tenemos la edad calculada.

In [8]:
df = data_non_zero
df.drop(columns='NUM_IDE', inplace=True)
df.drop(columns='FECHA_NACI', inplace=True)
df.drop(columns='id', inplace=True)
df.head()

,EPS,SEXO,DPTO,MUNI,DIAS_COMP,V8,Valor_Neto,DIAS_ESTAN_Neto,Edad,Grupo_Edad,Zona,Enfermedad
rownames,,,,,,,,,,,,
1184873,EPS002,F,50,1,360,B,39054.0,0.0,12.0,3,N,Sin Enfermedad
1587467,EPS013,M,76,1,336,C,312012.0,277.0,23.0,5,C,ENFERMEDADCARDIOVASCULAR_OTRA
753062,EPS017,F,11,1,333,B,14313.0,0.0,15.0,4,C,Sin Enfermedad
1802563,EPS002,M,76,1,84,B,0.0,0.0,13.0,3,C,Sin Enfermedad
1190692,EPS037,F,15,776,180,B,0.0,0.0,21.0,5,N,Sin Enfermedad


## Privatización

Para la privatización diferencial de los datos, es crucial adoptar enfoques distintos según el tipo de variable:

* **Variables Numéricas:** Utilizamos la mecánica de Laplace para agregar ruido a los datos. Este método introduce una perturbación aleatoria, distribuida según la función de Laplace, a cada valor numérico. La escala del ruido depende del parámetro epsilon, que determina el nivel de privacidad: un epsilon más pequeño aumenta la privacidad pero reduce la utilidad de los datos.

* **Variables Categóricas:** Aplicamos el método de Optimized Unary Encoding (OUE). Este enfoque convierte cada valor categórico en un vector binario y luego introduce ruido a cada bit del vector. Al igual que con Laplace, el nivel de ruido depende de epsilon, equilibrando la privacidad con la precisión de los datos.

In [ ]:
import pickle

# Definir las funciones de mecanismo de privacidad
def laplace_mechanism(value, epsilon):
    """Añade ruido laplaciano a un valor numérico."""
    scale = 1 / epsilon
    return value + np.random.laplace(0, scale)

def oue(data, column_name, epsilon):
    """Aplica Optimized Unary Encoding a una columna categórica."""
    categories = data[column_name].unique()
    def encode(category):
        p = 1 / (np.exp(epsilon) + 1)
        q = 1 - p
        encoded = [1 if np.random.rand() < p else 0 if cat == category else 1 if np.random.rand() < q else 0 for cat in categories]
        max_index = np.argmax(encoded)
        return categories[max_index]
    return data[column_name].apply(encode)

# Listas de columnas categóricas y numéricas
categorical_columns = ['EPS', 'SEXO', 'DPTO', 'MUNI', 'V8', 'Grupo_Edad', 'Zona', 'Enfermedad']
numeric_columns = ['Valor_Neto', 'DIAS_ESTAN_Neto', 'Edad']

# Valores de epsilon a usar
epsilons = [1.0]

# Aplicar mecanismos de privacidad y guardar DataFrames
for epsilon in epsilons:
    df_epsilon = df.copy()
    
    # Aplicar Laplace a columnas numéricas
    for col in numeric_columns:
        df_epsilon[col] = df_epsilon[col].apply(lambda x: laplace_mechanism(x, epsilon))
    
    # Aplicar OUE a columnas categóricas
    for col in categorical_columns:
        df_epsilon[col] = oue(df_epsilon, col, epsilon)
    
    # Guardar el DataFrame resultante como un archivo pickle
    pickle_filename = f'base_epsilon_{epsilon}.pkl'
    with open(pickle_filename, 'wb') as f:
        pickle.dump(df_epsilon, f)

    print(f'DataFrame para epsilon={epsilon} guardado como {pickle_filename}.')


Finalmente, privatizamos la base de datos usando un epsilon de 1. Sin embargo la función esta cosntruida para ejecutarse con cualquier epsilon